# IMPORT LIBRARIES

In [4]:
import pandas as pd
import numpy as np
import re
import spotipy

In [5]:
pitchfork = pd.read_csv('pitchfork.csv')
pitchfork.head()

,id,artist,content,genre,label,album,score,best_new_music,author,pub_date,year
0,22703,massive attack,"“Trip-hop” eventually became a ’90s punchline,...",electronic,virgin,mezzanine,9.3,0,nate patrin,2017-01-08,1998
1,22721,krallice,"Eight years, five albums, and two EPs in, the ...",metal,hathenter,prelapsarian,7.9,0,zoe camp,2017-01-07,2016
2,22659,uranium club,Minneapolis’ Uranium Club seem to revel in bei...,rock,static shock,all of them naturals,7.3,0,david glickman,2017-01-07,2016
3,22659,uranium club,Minneapolis’ Uranium Club seem to revel in bei...,rock,fashionable idiots,all of them naturals,7.3,0,david glickman,2017-01-07,2016
4,22661,kleenex,Kleenex began with a crash. It transpired one ...,rock,kill rock stars,first songs,9.0,1,jenn pelly,2017-01-06,2016


# GENRE vs. SCORE

In [6]:
genre_score = pitchfork[["genre","score","best_new_music"]]
genre_score_agg = ((genre_score.groupby('genre')).agg(['mean'])).sort_values(('best_new_music', 'mean'), ascending=False)
genre_score_agg

,score,best_new_music
,mean,mean
genre,,
pop/r&b,6.945943,0.076372
experimental,7.374098,0.075834
rock,7.005236,0.068946
rap,6.904837,0.059572
electronic,6.941103,0.056092
jazz,7.367910,0.046642
folk/country,7.221401,0.039490
global,7.368750,0.038690
